In [1]:
import pandas as pd

playground_df = pd.read_csv('train.csv')
playground_df.head(10)

,id,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,0,42,technician,married,secondary,no,7,no,no,cellular,25,aug,117,3,-1,0,unknown,0
1,1,38,blue-collar,married,secondary,no,514,no,no,unknown,18,jun,185,1,-1,0,unknown,0
2,2,36,blue-collar,married,secondary,no,602,yes,no,unknown,14,may,111,2,-1,0,unknown,0
3,3,27,student,single,secondary,no,34,yes,no,unknown,28,may,10,2,-1,0,unknown,0
4,4,26,technician,married,secondary,no,889,yes,no,cellular,3,feb,902,1,-1,0,unknown,1
5,5,24,admin.,single,secondary,no,1882,yes,no,cellular,20,apr,1010,3,-1,0,unknown,0
6,6,39,blue-collar,married,secondary,no,0,no,no,telephone,21,nov,90,1,-1,0,unknown,0
7,7,50,admin.,single,secondary,no,1595,no,no,telephone,31,jul,49,25,-1,0,unknown,0
8,8,46,blue-collar,married,primary,no,1463,no,no,cellular,4,aug,50,1,-1,0,unknown,0
9,9,39,management,divorced,tertiary,no,25,yes,no,cellular,8,may,119,1,-1,0,unknown,0


## 컬럼 설명 
- id             : id 번호 
- age            : 나이
- job            : 직업
- marital        : 결혼 여부
- education      : 교육 수준
- default        : 신용 불량 여부         
- balance        : 계좌 잔고
- housing        : 주택 보유 여부
- loan           : 대출 여부
- contact        : 마지막 접촉 수단
- day            : 마지막 접촉 일자
- month          : 마지막 접촉 월
- duration       : 마지막 접촉 통화 시간
- campaign       : 이번 캠페인 동안 연락한 횟수
- pdays          : 마지막 연락 후 경과일
- previous       : 이전 캠페인에서 연락 횟수
- poutcome       : 이전 캠페인의 결과
- y              : 예금 가입 여부

입력변수  : 
출력 변수 : y

In [2]:
playground_df.shape

(750000, 18)

In [3]:
playground_df['job'].value_counts()

job
management       175541
blue-collar      170498
technician       138107
admin.            81492
services          64209
retired           35185
self-employed     19020
entrepreneur      17718
unemployed        17634
housemaid         15912
student           11767
unknown            2917
Name: count, dtype: int64

In [4]:
playground_df.isna().sum()

id           0
age          0
job          0
marital      0
education    0
default      0
balance      0
housing      0
loan         0
contact      0
day          0
month        0
duration     0
campaign     0
pdays        0
previous     0
poutcome     0
y            0
dtype: int64

In [5]:
playground_df[['previous','campaign']].corr()

,previous,campaign
previous,1.000000,-0.026707
campaign,-0.026707,1.000000


In [6]:
playground_df['poutcome'].value_counts().sort_values(ascending=False)

poutcome
unknown    672450
failure     45115
success     17691
other       14744
Name: count, dtype: int64

In [7]:
playground_df['pdays'].value_counts()

pdays
-1      672434
 182      2515
 92       2275
 183      2074
 181      1698
         ...  
 735         1
 611         1
 714         1
 54          1
 376         1
Name: count, Length: 596, dtype: int64

In [8]:
playground_df.sort_values(by='age', ascending=False)

,id,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
283611,283611,95,retired,married,secondary,no,4984,no,no,telephone,26,oct,88,1,-1,0,unknown,0
557988,557988,95,retired,divorced,primary,no,852,no,no,telephone,1,apr,223,2,-1,0,unknown,1
551450,551450,95,retired,married,secondary,no,4657,no,no,telephone,31,mar,466,1,-1,0,unknown,1
164587,164587,94,retired,divorced,primary,no,1388,no,no,cellular,12,nov,212,1,-1,0,unknown,1
449210,449210,94,retired,married,unknown,no,12039,no,no,telephone,30,apr,516,4,-1,0,unknown,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
428100,428100,18,student,single,secondary,no,608,no,no,cellular,1,jun,425,1,-1,0,unknown,1
138704,138704,18,student,single,secondary,no,2143,no,no,cellular,6,jul,265,3,178,5,success,1
453343,453343,18,student,single,secondary,no,608,no,no,cellular,4,may,226,1,-1,0,unknown,1
445783,445783,18,admin.,single,secondary,no,696,yes,yes,cellular,13,may,474,2,-1,0,unknown,0


In [9]:
pd.DataFrame(playground_df[playground_df['pdays']==-1])

,id,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,0,42,technician,married,secondary,no,7,no,no,cellular,25,aug,117,3,-1,0,unknown,0
1,1,38,blue-collar,married,secondary,no,514,no,no,unknown,18,jun,185,1,-1,0,unknown,0
2,2,36,blue-collar,married,secondary,no,602,yes,no,unknown,14,may,111,2,-1,0,unknown,0
3,3,27,student,single,secondary,no,34,yes,no,unknown,28,may,10,2,-1,0,unknown,0
4,4,26,technician,married,secondary,no,889,yes,no,cellular,3,feb,902,1,-1,0,unknown,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
749994,749994,31,housemaid,single,secondary,no,594,no,no,cellular,4,feb,137,1,-1,0,unknown,0
749995,749995,29,services,single,secondary,no,1282,no,yes,unknown,4,jul,1006,2,-1,0,unknown,1
749996,749996,69,retired,divorced,tertiary,no,631,no,no,cellular,19,aug,87,1,-1,0,unknown,0
749997,749997,50,blue-collar,married,secondary,no,217,yes,no,cellular,17,apr,113,1,-1,0,unknown,0


## 카이제곱 검정 시행

In [10]:
chi = ['marital','education','housing','loan','contact','month','poutcome']

In [11]:
import scipy.stats as stats
from scipy.stats import chi2_contingency

for i in chi:
    ct = pd.crosstab(playground_df[i],playground_df['y'])
    chi2, p, _, _ = chi2_contingency(ct)
    print(f'== {i} vs y ==')
    print(ct)
    print(f'Chi2: {chi2:.3f}, p-value: {p:.8f}')
    print('\n')

== marital vs y ==
y              0      1
marital                
divorced   66105   8302
married   431783  48976
single    161624  33210
Chi2: 6210.479, p-value: 0.00000000


== education vs y ==
y               0      1
education               
primary     91241   8269
secondary  359309  42374
tertiary   190504  37004
unknown     18458   2841
Chi2: 6007.856, p-value: 0.00000000


== housing vs y ==
y             0      1
housing               
no       279174  59538
yes      380338  30950
Chi2: 17691.305, p-value: 0.00000000


== loan vs y ==
y          0      1
loan               
no    560281  84742
yes    99231   5746
Chi2: 4997.963, p-value: 0.00000000


== contact vs y ==
y               0      1
contact                 
cellular   410455  76200
telephone   27379   4339
unknown    221678   9949
Chi2: 19179.184, p-value: 0.00000000


== month vs y ==
y           0      1
month               
apr     31582   9737
aug    114406  14453
dec      1007   1062
feb     29833   7778
jan 

데이터가 많아서 p-value값이 0이 나온 것 같긴 함                                       
그래도 통계적 유의성만 본다면 범주형 컬럼중에 제외시켜야 할 컬럼은 없는 것 같음

In [12]:
map_YN = ['default','housing','loan']
for col in map_YN:
    playground_df[col] = playground_df[col].map({
    'no' : 0,
    'yes' : 1
})

In [13]:
playground_df

,id,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,0,42,technician,married,secondary,0,7,0,0,cellular,25,aug,117,3,-1,0,unknown,0
1,1,38,blue-collar,married,secondary,0,514,0,0,unknown,18,jun,185,1,-1,0,unknown,0
2,2,36,blue-collar,married,secondary,0,602,1,0,unknown,14,may,111,2,-1,0,unknown,0
3,3,27,student,single,secondary,0,34,1,0,unknown,28,may,10,2,-1,0,unknown,0
4,4,26,technician,married,secondary,0,889,1,0,cellular,3,feb,902,1,-1,0,unknown,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
749995,749995,29,services,single,secondary,0,1282,0,1,unknown,4,jul,1006,2,-1,0,unknown,1
749996,749996,69,retired,divorced,tertiary,0,631,0,0,cellular,19,aug,87,1,-1,0,unknown,0
749997,749997,50,blue-collar,married,secondary,0,217,1,0,cellular,17,apr,113,1,-1,0,unknown,0
749998,749998,32,technician,married,secondary,0,-274,0,0,cellular,26,aug,108,6,-1,0,unknown,0


In [14]:
playground_df = pd.get_dummies(columns=['job','marital','education','contact','month','poutcome'],data=playground_df)

In [15]:
playground_df.shape

(750000, 50)

In [16]:
X = playground_df.drop(columns='id')
y = playground_df['y']

In [17]:
from sklearn.model_selection import train_test_split

X_train,X_val,y_train,y_val = train_test_split(X,y, test_size=0.2)

In [20]:
from lazypredict.Supervised import LazyClassifier

clf = LazyClassifier()
models, predictions = clf.fit(X_train,X_val,y_train,y_val)
print(models.head(5))

  0%|          | 0/32 [00:00<?, ?it/s]

[LightGBM] [Info] Number of positive: 72362, number of negative: 527638
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.014541 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 967
[LightGBM] [Info] Number of data points in the train set: 600000, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.120603 -> initscore=-1.986729
[LightGBM] [Info] Start training from score -1.986729
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best ga

In [21]:
import optuna
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve
from sklearn.model_selection import cross_val_score, StratifiedKFold

def objective(trial):
    n_estimators = trial.suggest_int("n_estimators", 50, 500)
    learning_rate = trial.suggest_float("learning_rate", 0.01,1.0, log=True)
    max_depth = trial.suggest_int("max_depth", 1, 20)

    base_estimator = DecisionTreeClassifier(max_depth=max_depth)

    model = AdaBoostClassifier(
    estimator=base_estimator,
    n_estimators=n_estimators,
    learning_rate = learning_rate,
    random_state=2020158011
    )

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=2020158011)
    scores = cross_val_score(model,X_train,y_train, cv=cv, scoring='roc_auc')
    return scores.mean()

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

print("best trial : ")
trial = study.best_trial
print(f" AUC score : {trial.value}")
print(f" Params   : {trial.params}")

[I 2025-08-08 14:11:08,494] A new study created in memory with name: no-name-c6634668-0487-4b93-8276-c5cec366b3ee
[W 2025-08-08 14:11:10,057] Trial 0 failed with parameters: {'n_estimators': 181, 'learning_rate': 0.15077231974341201, 'max_depth': 9} because of the following error: The value nan is not acceptable.
[W 2025-08-08 14:11:10,057] Trial 0 failed with value np.float64(nan).
[I 2025-08-08 14:11:13,701] Trial 1 finished with value: 1.0 and parameters: {'n_estimators': 390, 'learning_rate': 0.13765834454267595, 'max_depth': 13}. Best is trial 1 with value: 1.0.
[I 2025-08-08 14:11:17,515] Trial 2 finished with value: 1.0 and parameters: {'n_estimators': 146, 'learning_rate': 0.06571751180406651, 'max_depth': 19}. Best is trial 1 with value: 1.0.
[I 2025-08-08 14:11:21,224] Trial 3 finished with value: 1.0 and parameters: {'n_estimators': 389, 'learning_rate': 0.8080801660780079, 'max_depth': 12}. Best is trial 1 with value: 1.0.
[I 2025-08-08 14:11:24,926] Trial 4 finished with v

ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\user\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\model_selection\_validation.py", line 859, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
    ~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\user\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\base.py", line 1363, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "C:\Users\user\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\ensemble\_weight_boosting.py", line 130, in fit
    X, y = validate_data(
           ~~~~~~~~~~~~~^
        self,
        ^^^^^
    ...<6 line

In [ ]:
best_params = study.best_params
print(best_params)

In [ ]:
best_model = AdaBoostClassifier(**best_params)
best_model.fit(X,y)

## 테스트 데이터 전처리